In [ ]:
import numpy as np
import random
import statistics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
class conflict_organism:
    def __init__(self, strength):
        self.strength = strength

    def __str__(self):
        return f"{self.strength}"

    def breed(self):
        global conflict_organism_pop
        strength1 = np.round(self.strength + np.random.randn()*1.5)
        if np.random.rand()>len(conflict_organism_pop)/1000:
            conflict_organism_pop.append(conflict_organism(strength1))

In [ ]:
class organism:
    def __init__(self, strength, period, gender, gender_culture, family):
        self.strength = strength
        self.period = period
        self.gender = gender
        self.gender_culture = gender_culture
        self.family=family

    def __str__(self):
            return f"{self.gender},{self.strength}({self.period}, {self.gender_culture})"

In [ ]:
#ranking
def reproduce():
    global m_r
    global population
    global conflict_organism_pop
    global child_population
    global male_pop


    child_population=[]
    
    female_pop =[p for p in population if p.gender == 'female']
    male_pop_temp=[p for p in population if p.gender == 'male']

    for i in male_pop_temp:
        i.family=[]

    
    herm_pop = [p for p in population if p.gender == 'hermaphrodite']
    male_pop= sorted(male_pop_temp, key=lambda x: x.strength, reverse=False)
    male_pop_positive=[p for p in male_pop if p.gender_culture > 0]
    male_pop_negative=[p for p in male_pop if p.gender_culture <= 0]

    population=[]
    
    if len(male_pop)>0:
        
        index=0
        r=0.5
        male_pop_list_positive=[]
        for i in male_pop_positive:
#            selection_formula = 1/(len(male_pop_positive))
#            selection_formula = ((pow(r, index)*(1-r))/(1-pow(r, len(male_pop_positive))))
            index+=1
            selection_formula = 2*index/(len(male_pop_positive)*(len(male_pop_positive)+1))
            male_pop_list_positive.append(selection_formula)

        index=0
        male_pop_list_negative=[]
        for i in male_pop_negative:
#            selection_formula = 1/(len(male_pop_negative))
#            selection_formula = ((pow(r, index)*(1-r))/(1-pow(r, len(male_pop_negative))))
            index+=1
            selection_formula = 2*index/(len(male_pop_negative)*(len(male_pop_negative)+1))
            male_pop_list_negative.append(selection_formula)

            
        while len(female_pop)>0:
            
            random_org1=np.random.choice(range(len(female_pop)))
            org1=female_pop[random_org1]

            if org1.gender_culture >0:

                if len(male_pop_positive) >0:
                    random_org2=np.random.choice(range(len(male_pop_positive)), p=male_pop_list_positive)
                    org2=male_pop_positive[random_org2]
                else:
                    random_org2=np.random.choice(range(len(male_pop_negative)), p=male_pop_list_negative)
                    org2=male_pop_negative[random_org2]

            else:
                if len(male_pop_negative) >0:
                    random_org2=np.random.choice(range(len(male_pop_negative)), p=male_pop_list_negative)
                    org2=male_pop_negative[random_org2]

                else:
                    random_org2=np.random.choice(range(len(male_pop_positive)), p=male_pop_list_positive)
                    org2=male_pop_positive[random_org2]

             
    
            gender1='female'
            gender2='male'
            
            if np.random.rand()<m_r:
                gender1='male'
                if np.random.rand()<0.5:
                    gender1='hermaphrodite'
            if np.random.rand()<m_r:
                gender2='female'
                if np.random.rand()<0.5:
                    gender2='hermaphrodite'

            gen_cul1 = org1.gender_culture+np.random.randn()
            gen_cul2 = org2.gender_culture+np.random.randn()

    
            str1 = np.round(org1.strength + np.random.randn(), decimals=2)
            str2 = np.round(org2.strength + np.random.randn(), decimals=2)


            fam=org1.family
            org2.family.append(fam)

            child1 = organism(str1, 1, gender1, gen_cul1, fam)
            child2 = organism(str2, 1, gender2, gen_cul2, fam)

                
            if np.random.rand()<0.5:
                child_population.append(child1)
            else:
                child_population.append(child2)

            population.append(org1)
    
            female_pop.remove(female_pop[random_org1])


    for single_parent in herm_pop:
    
        gender1='hermaphrodite'

        if np.random.rand()<m_r:
            if np.random.rand()<0.5:
                gender1='male'
            else:
                gender1='female'

    
        strength1 = np.round(single_parent.strength + np.random.randn(), decimals=2)

        gen_cul1 = single_parent.gender_culture
        
        fam=single_parent.family
    
        child1 = organism(strength1, 1, gender1, gen_cul1, fam)
        child_population.append(child1)


    if len(female_pop)>0:
        for i in female_pop:
            population.append(i)

    for i in male_pop:
        population.append(i)
            
    for i in herm_pop:
        population.append(i)
    

In [ ]:
def hunt(org):
    global conflict_organism_pop
    global population

    if len(conflict_organism_pop)>0:
        random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
        if org.strength >= conflict_organism_pop[random_conflict_organism].strength+np.random.randn()/50:
            conflict_organism_pop.remove(conflict_organism_pop[random_conflict_organism])
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
def single_parent_hunt(org, org_family):
    global conflict_organism_pop
    global population
    global child_population
    global surviving_herm_children

    if len(conflict_organism_pop)>0:
        child =[p for p in child_population if p.family == org_family.family]
    
        if len(child)>0:
            random_conflict_organism=np.random.choice(range(len(conflict_organism_pop)))
    
            if hunt(org) == 1:
                population.append(child[0])
                if org.gender == 'hermaphrodite':
                    surviving_herm_children+=1
                    
                if hunt(org) == 1:
                    return 1
                else:
                    return 0
            else:
                return 0
        else:
            if hunt(org) == 1:
                return 1
            else:
                return 0
    else:
        return 0        

In [ ]:
def remove_father(org):
    global male_pop
    global population

    target_gender_culture = org.gender_culture

    for i in male_pop:
        if i.gender_culture == target_gender_culture:
            male_pop.remove(i)
            break
        
    for i in population:
        if i.gender_culture == target_gender_culture and i.gender == 'male':
            population.remove(i)
            break            

In [ ]:
def hunt_and_feed():
    global conflict_organism_pop
    global population
    global child_population
    global male_pop
    global child_population_ratio_before
    global population_ratio_before
    global child_population_ratio_after
    global population_ratio_after
    global surviving_herm_children
    global expected_share


    herm_pop_temp=[p for p in population if p.gender == 'hermaphrodite']
    if len(herm_pop_temp) > 0:
        population_ratio_before=len(herm_pop_temp)/len(population)
        child_population_ratio_before = len(herm_pop_temp)/len(child_population)
    else:
        population_ratio_before = 0
        child_population_ratio_before = 0

    surviving_herm_children = 0

    
    hunting_population=population.copy()
    random.shuffle(hunting_population)
    population=[]

    for adult in hunting_population:

        if adult.gender == 'male':

            if hunt(adult) == 1:
                population.append(adult)
            else:
                remove_father(adult)
                

        if adult.gender == 'hermaphrodite': 
            if single_parent_hunt(adult, adult) == 1:
                population.append(adult)

        if adult.gender == 'female':
            father=0
            for i in male_pop:
                if adult.family in i.family:
                    father=i
                    
            if father==0:
                if single_parent_hunt(adult, adult) == 1:
                    population.append(adult)
                    
            elif adult.gender_culture+father.gender_culture<0:
                if single_parent_hunt(adult, adult) == 1:
                    population.append(adult)

            else:
                if single_parent_hunt(father, adult) == 1:
                    population.append(adult)
                else:
                    remove_father(father)
                    if hunt(adult) == 1:
                        population.append(adult) 
    
  
    adult_population=[p for p in population if p.period>1]
    herm_pop_temp=[p for p in adult_population if p.gender == 'hermaphrodite']
    if len(herm_pop_temp) > 0:
        population_ratio_after=len(herm_pop_temp)/len(adult_population)
    else:
        population_ratio_after = 0

    child_pop_temp=[p for p in population if p.period == 1]     
    if len(child_pop_temp) > 0:
        child_population_ratio_after = surviving_herm_children/len(child_pop_temp)
    else:
        child_population_ratio_after= 1

    
    for org in population:
        org.period+=1
    population=[p for p in population if p.period <= 4]

    random.shuffle(conflict_organism_pop)
    for conflict_org in conflict_organism_pop:
        conflict_org.breed()

    k=0
    for org in population:
        org.family=k
        k+=1

In [ ]:
#total number of times you run the simulation
number_of_total_iter = 120
#number of generations per regression
number_of_iter=2000
#mutation rate
m_r=0.05

fig, axes = plt.subplots(int(number_of_total_iter), 3, figsize=(18, number_of_total_iter*5))
axes = axes.flatten()

invalid_iters=0
positives=0
negatives=0
positive_shares=[]
neutral_shares=[]
negative_shares=[]
average_cult=[]
max_positives_share=0
max_neutrals_share=0
max_negatives_share=0
x=0
while x<number_of_total_iter*3:

    valid=1
    share_plot=[]
    culture_plot_males=[]
    culture_plot_females=[]
    total_culture_plot=[]
    population_plot=[]
    adult_change_plot=[]
    child_change_plot=[]
    n = 100
    n_conflict_organism=300
    population=[]
    conflict_organism_pop=[]
    share0=0
    
    for i in range(n):
        strength=np.round(10+np.random.randn()*5)
        age=np.round(1+np.random.exponential(2))
        gen_cul = np.round(np.random.randn()/10, decimals=5)
    
        if age<2:
            age=2
        if age>4:
            age=4
        if np.random.rand()<m_r:
            share0+=100/n
            if np.random.rand()<0.5:
                gender='male'
            else:
                gender='female'
        else:
            gender='hermaphrodite'
            
        new_organism=organism(strength, age, gender, gen_cul, i)
        population.append(new_organism)
    
    for i in range(n_conflict_organism):
        strength=np.round(12+np.random.randn()*5)
        if strength<1:
            strength=1
        new_conflict_organism=conflict_organism(strength)
        conflict_organism_pop.append(new_conflict_organism)
    
    share_plot.append(share0)
    number=0
    long_term_trend=[]
    long_long_term_trend=[]
    
    
    while number<number_of_iter:
        reproduce()
        hunt_and_feed()
        if len(population)<=0:
            valid=0
            break
        if len(conflict_organism_pop)<=0:
            valid=0
            break
    
        str_conflict_organism=0
        for i in conflict_organism_pop:
            str_conflict_organism+=i.strength
    
        males=[p for p in population if p.gender == 'male']
        females=[p for p in population if p.gender == 'female']
        hermaphrodites=[p for p in population if p.gender == 'hermaphrodite']
        st_males=0
        total_culture_males=0
        total_culture_females=0
    
        for i in males:
            st_males+=i.strength
            total_culture_males+=i.gender_culture 
        st_females=0
        for i in females:
            st_females+=i.strength
            total_culture_females+=i.gender_culture 
        st_hermaphrodites=0
        for i in hermaphrodites:
            st_hermaphrodites+=i.strength
        
        male_share=np.round(len(males)/len(population), decimals=3)
        female_share=np.round(len(females)/len(population), decimals=3)
        share=np.round(100*(len(males)+len(females))/len(population), decimals=1)
        long_term_trend.append(share)
        long_long_term_trend.append(share)
        share_plot.append(share)
            
        population_plot.append(len(population))
    
    
    
        if len(males)+len(females)>0:
            total_culture_plot.append((total_culture_males+total_culture_females)/(len(males)+len(females)))
        else:
            total_culture_plot.append(0)
    
        if population_ratio_before>population_ratio_after:
            adult_change_plot.append(1)
        elif population_ratio_before == population_ratio_after:
            adult_change_plot.append(0)
        else:
            adult_change_plot.append(-1)
    
        if child_population_ratio_before>child_population_ratio_after:
            child_change_plot.append(1)
        elif child_population_ratio_before == child_population_ratio_after:
            child_change_plot.append(0)
        else:
            child_change_plot.append(-1)
            
        number+=1

    if valid ==0:
        invalid_iters+=1
        print('invalid iterations', invalid_iters)
    else:
        final_share=np.round(statistics.mean(share_plot[-1001:-1]), decimals=3)

        cm = confusion_matrix(adult_change_plot[-1001:-1], child_change_plot[-1001:-1])
        
        count_positive_child = child_change_plot[-1001:-1].count(1)
        count_negative_child = child_change_plot[-1001:-1].count(-1)
        count_zero_child = child_change_plot[-1001:-1].count(0)

        count_positive_adult = adult_change_plot[-1001:-1].count(1)
        count_negative_adult = adult_change_plot[-1001:-1].count(-1)
        count_zero_adult = adult_change_plot[-1001:-1].count(0)

        if count_zero_child == 0 and count_zero_adult==0:
            labels = [-1, 1]
        else:
            labels = [-1, 0, 1]
                
        if count_positive_child>count_negative_child and count_positive_adult>count_negative_adult:
            color = 'Greens'
        elif count_positive_child<count_negative_child and count_positive_adult<count_negative_adult:
            color = 'Reds'
        else:
            color='YlOrBr'
        
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(ax=axes[x], cmap=color, values_format='d', colorbar=False)
        axes[x].set_title(str(int(x/3+1)) + '. Hunting performance')
        axes[x].set_xlabel('More Surviving Children than Expected', fontsize=8)
        axes[x].set_ylabel('More Surviving Adults than Expected', fontsize=8)
        
        if statistics.mean(total_culture_plot[-1001:-1])>1.5:
            positives+=1
            positive_shares.append(final_share)
            if final_share>max_positives_share:
                max_positives_share=final_share
            color_culture = 'green'
        elif statistics.mean(total_culture_plot[-1001:-1])<-1.5:
            negatives+=1
            negative_shares.append(final_share)
            if final_share>max_negatives_share:
                max_negatives_share=final_share
            color_culture = 'red'
        else:
            neutral_shares.append(final_share)
            if final_share>max_neutrals_share:
                max_neutrals_share=final_share
            color_culture = 'orange'
        
        if final_share>14.205:
            color_share = 'green'
        elif final_share<12.758:
            color_share = 'red'
        else:
            color_share = 'orange'

        average_cult.append(np.round(statistics.mean(total_culture_plot[-1001:-1]), decimals=3))
            
        
        axes[x+1].plot(total_culture_plot[-1001:-1], color=color_culture)
        axes[x+1].set_title(str(int(x/3+1)) + '. Average culture of non-hermaphrodites')
        axes[x+1].set_xlabel('Average Culture =' + str(np.round(statistics.mean(total_culture_plot[-1001:-1]), decimals=3)), fontsize=8)
        
        axes[x+2].plot(share_plot[-1001:-1], color=color_share)
        axes[x+2].set_title(str(int(x/3+1)) + '. Share of non-hermaphrodites (in %)')
        axes[x+2].set_xlabel('Average Share =' + str(final_share), fontsize=8)
        
        print(1+x/3)
        x+=3

print('number of invalid iterations:', invalid_iters)
if positives>0:
    print('number of positives:', positives)
    print('average share (positive culture):', np.round(statistics.mean(positive_shares), decimals=3))
    print('maximum share (positive culture):', max_positives_share)
if number_of_total_iter>negatives+positives:
    print('number of neutrals:', number_of_total_iter-negatives-positives)
    print('average share (neutral culture):', np.round(statistics.mean(neutral_shares), decimals=3))
    print('maximum share (neutral culture):', max_neutrals_share)
if negatives > 0:
    print('number of negatives:', negatives)
    print('average share (negative culture):', np.round(statistics.mean(negative_shares), decimals=3))
    print('maximum share (negative culture):', max_negatives_share)

plt.savefig('culture_test.png')
    
plt.show()

In [ ]:
from scipy import stats
alpha = 0.01

In [ ]:
mean_pos = np.mean(positive_shares)
std_pos = np.std(positive_shares, ddof=1)
sem_pos = std_pos / np.sqrt(positives)
df_pos = positives - 1
t_crit_pos = stats.t.ppf(1 - alpha/2, df_pos)
margin_of_error_pos = t_crit_pos * sem_pos
ci_lower_pos = mean_pos - margin_of_error_pos
ci_upper_pos = mean_pos + margin_of_error_pos

print('[', ci_lower_pos, ', ', ci_upper_pos, ']')

In [ ]:
mean_nut = np.mean(neutral_shares)
std_nut = np.std(neutral_shares, ddof=1)
sem_nut = std_nut / np.sqrt(number_of_total_iter-negatives-positives)
df_nut = number_of_total_iter-negatives-positives - 1
t_crit_nut = stats.t.ppf(1 - alpha/2, df_nut)
margin_of_error_nut = t_crit_nut * sem_nut
ci_lower_nut = mean_nut - margin_of_error_nut
ci_upper_nut = mean_nut + margin_of_error_nut

print('[', ci_lower_nut, ', ', ci_upper_nut, ']')

In [ ]:
mean_neg = np.mean(negative_shares)
std_neg = np.std(negative_shares, ddof=1)
sem_neg = std_neg / np.sqrt(negatives)
df_neg = negatives - 1
t_crit_neg = stats.t.ppf(1 - alpha/2, df_neg)
margin_of_error_neg = t_crit_neg * sem_neg
ci_lower_neg = mean_neg - margin_of_error_neg
ci_upper_neg = mean_neg + margin_of_error_neg

print('[', ci_lower_neg, ', ', ci_upper_neg, ']')

In [ ]:
mean_cult = np.mean(average_cult)
std_cult = np.std(average_cult, ddof=1)
sem_cult = std_cult / np.sqrt(len(average_cult))
df_cult = len(average_cult) - 1
t_crit_cult = stats.t.ppf(1 - alpha/2, df_cult)
margin_of_error_cult = t_crit_cult * sem_cult
ci_lower_cult = mean_cult - margin_of_error_cult
ci_upper_cult = mean_cult + margin_of_error_cult

print('[', ci_lower_cult, ', ', ci_upper_cult, ']')

In [ ]:
counts, bin_edges = np.histogram(average_cult, bins='auto')
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
sorted_idx = np.argsort(counts)[::-1]

plt.bar(bin_centers[sorted_idx], counts[sorted_idx], width=0.9*(bin_edges[1] - bin_edges[0]))
plt.xlabel('Values (binned)' + '\n \n mean = ' + str(np.round(mean_cult, decimals=3)) + ';  std = ' + str(np.round(std_cult, decimals=3)) +
    '; \n CI = [' + str(np.round(ci_lower_cult, decimals=3)) + ' , ' + str(np.round(ci_upper_cult, decimals=3)) + ']')
plt.ylabel('Frequency')
plt.title('Average Culture (120 iterations)')

plt.savefig('hist_herm.png', bbox_inches='tight')

plt.show()